In [2]:
import numpy as np
import os
import json
import csv
import unicodedata
import re
from string import printable
import pandas as pd
import re
from functools import partial
from greeklish.converter import Converter
#test1

In [3]:
def remove_emoji(df):
  return df.applymap(lambda y: ''.join(filter(lambda x: x in printable, y)))

myconverter = Converter(max_expansions=1)
final_df = pd.DataFrame()

fix_mojibake_escapes = partial(
     re.compile(rb'\\u00([\da-f]{2})').sub,
     lambda m: bytes.fromhex(m.group(1).decode()))

In [4]:

dir = "inbox"
folders = os.listdir(dir)
path_list = []
paths_senders = {}
for folder in folders:
    for file in os.listdir(os.path.join("inbox",folder)):
        if file.startswith("message"):
            path = os.path.join(dir,folder,file)
            try:
                with open(path, 'rb') as file:
                    repaired = fix_mojibake_escapes(file.read())
                    data = json.loads(repaired.decode('utf8'))

                    sender_name = data["participants"][0]["name"]
                    messages = data["messages"]
                    new_conversation = []
                    for k in messages:
                        if "content" in k.keys():
                            new_conversation.append(k)
                    pairs = []
                    message = [] #message pair


                    for m in new_conversation[::-1]:
                        if len(message) == 0 or len(message) == 2:
                            if m["sender_name"] != "Antonis Parlapanis":
                                message = []
                                message.append(m["content"])
                        elif len(message) == 1:
                            if m["sender_name"] != "Antonis Parlapanis":
                                message[0] = message[0]+ " " + m["content"] #enwnei duo munhmata sth seira
                            else:
                                message.append(m["content"])
                        else:
                            if m["sender_name"] == "Antonis Parlapanis":
                                message[1] = message[1] + " " + m["content"]
                        pairs.append(message)
                    new_pair = []
                    for pair in pairs:
                        if len(pair) == 2:
                            new_pair.append(pair)


                    me = []
                    sender = []

                    for pair in new_pair:
                        sender.append(pair[0])
                        me.append(pair[1])

                    conv_df = pd.DataFrame()
                    conv_df["sender"] = np.array(sender)
                    conv_df["me"] = np.array(me)
                    conv_df = conv_df.drop_duplicates(keep="first")

                    # conv_df = remove_emoji(conv_df)
                    final_df = pd.concat([conv_df,final_df],axis=0)




            except json.decoder.JSONDecodeError:
                print("There was a problem accessing the equipment data.")




final_df.drop(final_df[final_df["sender"]=="  "].index,inplace=True)
final_df.drop(final_df[final_df["me"]=="  "].index,inplace=True)


#
final_df["sender"] = final_df["sender"].apply(lambda x: myconverter.convert(x))
final_df["me"] = final_df["me"].apply(lambda x: myconverter.convert(x))




# Lowercase, trim, and remove non-letter characters
def normalize(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

final_df["sender"] = final_df["sender"].apply(lambda x: normalize(x[0]))
final_df["me"] = final_df["me"].apply(lambda x: normalize(x[0]))

print(final_df.head(20))


final_df.to_csv('out.csv', sep='\t', encoding='utf-8')


There was a problem accessing the equipment data.
                                               sender  \
0                       eimai poly thymwmenh mazi sou   
3                                          pou eisai    
5                                                geia   
7                            kalaaaaa auto einai egw    
11                       apo thn gefyra ths enorganhs   
13                             ekana xronia pou eisia   
16                               xaxaxaxaxa gkomenaki   
24       re blhma danai set her own nickname to ss .    
29                          ela twra etsi de fainetai   
33          egw eixa gynaikologo alliws tha koimomoun   
35                              kai ti tha kaneis smr   
39                                               pote   
41                                        okkk antwnh   
44                                  forema h mplouza    
47                                        panagia mou   
49                  mias filhs mou gia

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer()
tokenizer.fit_on_texts(final_df['sender'])
train = tokenizer.texts_to_sequences(final_df['sender'])

x_train = pad_sequences(train)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(final_df['me'])


In [6]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

371
number of unique words :  9029
output length:  4775


In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model

#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=20)


Epoch 1/20
204/204 [==============================] - 19s 91ms/step - loss: 8.4483 - accuracy: 0.0312
Epoch 2/20
204/204 [==============================] - 19s 91ms/step - loss: 8.1426 - accuracy: 0.0415
Epoch 3/20
204/204 [==============================] - 19s 94ms/step - loss: 7.9748 - accuracy: 0.0435
Epoch 4/20
204/204 [==============================] - 19s 92ms/step - loss: 7.7113 - accuracy: 0.0435
Epoch 5/20
204/204 [==============================] - 19s 93ms/step - loss: 7.0510 - accuracy: 0.0567
Epoch 6/20
204/204 [==============================] - 19s 94ms/step - loss: 6.2774 - accuracy: 0.1082
Epoch 7/20
204/204 [==============================] - 20s 97ms/step - loss: 5.6139 - accuracy: 0.1798
Epoch 8/20
204/204 [==============================] - 19s 94ms/step - loss: 5.0764 - accuracy: 0.2414
Epoch 9/20
204/204 [==============================] - 19s 94ms/step - loss: 4.6357 - accuracy: 0.3007
Epoch 10/20
204/204 [==============================] - 19s 94ms/step - loss: 4.264

In [1]:
#chatting
import random
import string
while True:
  texts_p = []
  prediction_input = input('steile ston antwnh : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print(response_tag)

steile ston antwnh : geia


NameError: name 'tokenizer' is not defined